In [1]:
import pickle 
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data = data.drop(columns=['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Lable Encoding Categorical varaibles
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
lable_encoder_gender = LabelEncoder()
data['Gender'] = lable_encoder_gender.fit_transform(data['Gender'])

In [5]:
## One Hot Encoding of Geography
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
geo_encoded = one_hot_encoder.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [7]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
## split the data into features and target
x = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [9]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [10]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
# with open('lable_encoder_gender.pkl','wb') as file:
#     pickle.dump(lable_encoder_gender,file)

# with open('one_hot_encoder_geo.pkl','wb') as file:
#     pickle.dump(one_hot_encoder,file)
    
# with open('scaler.pkl','wb') as file:
#     pickle.dump(scaler,file)

## save all the pickle files 

## ANN Regression Problem Statement


In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [13]:
## build our ANN Model
model = Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)), ## HL1 Connected to input layer
    Dense(32,activation='relu'), ## HL2
    Dense(1) ## Output layer
])

In [14]:
## Compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

## Set Up TensorBoard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [18]:
## Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [19]:
## train The Model
history = model.fit(
    x_train,y_train,
    validation_data=(x_test,y_test),
    epochs = 100,
    callbacks = [early_stopping_callback,tensorflow_callback]
)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 100386.6094 - mae: 100386.6094 - val_loss: 98549.2266 - val_mae: 98549.2266
Epoch 2/100
250/250 [==============================] - 0s 718us/step - loss: 99733.1172 - mae: 99733.1172 - val_loss: 97202.4609 - val_mae: 97202.4609
Epoch 3/100
250/250 [==============================] - 0s 669us/step - loss: 97343.0703 - mae: 97343.0703 - val_loss: 93658.6484 - val_mae: 93658.6484
Epoch 4/100
250/250 [==============================] - 0s 647us/step - loss: 92577.5703 - mae: 92577.5703 - val_loss: 87660.6250 - val_mae: 87660.6250
Epoch 5/100
250/250 [==============================] - 0s 659us/step - loss: 85520.3906 - mae: 85520.3906 - val_loss: 79755.0000 - val_mae: 79755.0000
Epoch 6/100
250/250 [==============================] - 0s 764us/step - loss: 76971.3359 - mae: 76971.3359 - val_loss: 71136.2734 - val_mae: 71136.2734
Epoch 7/100
250/250 [==============================] - 0s 675us/step - loss: 68367.3047 - mae:

In [20]:
%load_ext tensorboard

In [24]:
%tensorboard --logdir regressionlogs/fit/20250522-211628

Reusing TensorBoard on port 6007 (pid 36514), started 0:00:03 ago. (Use '!kill 36514' to kill it.)

In [25]:
## evaluate model on the test data
test_loss, test_mae = model.evaluate(x_test,y_test)
print(f"test MAE : {test_mae}")

63/63 [==============================] - 0s 395us/step - loss: 50270.9531 - mae: 50270.9531
test MAE : 50270.953125


In [26]:
model.save('regression_model.h5')

/Users/vaibhavkale/Desktop/DL_Projects/ANN_Classification/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
